In [358]:
import os
import glob
import json
import pandas as pd
from tqdm import tqdm_notebook as tqdm

In [324]:
farfetch_scrape_annos = glob.glob('/data/farfetch-scrape-v2/annotations/*.json')

In [338]:
cat2keywords = {
    'bag': ['bag', 'bags', 'tote', 'backpack', 'clutch', 
            'handbag', 'purse'],
    'belt': ['belt'],
    'outer': ['coat', 'jacket', 'suit', 'blazers', 'sweatshirt', 
              'hoodie', 'vest', 'blazer', 'raincoat', 'gilet',
             'sweater', 'hood', 'poncho', 'waistcoat', 'jumper',
             'cardigan', 'pullover', 'jumpsuit', 'overcoat', 'parka',
             'sweatshir'],
    'dress': ['dress', 'gown'],
    'sunglasses': ['glasses', 'sunglasses'],
    'pants': ['pants', 'jeans', 'trousers', 'sweatpants', 'pants', 
             'chinos', 'leggings', 'sweater-trousers', 'tights', 
             'trouser', 'joggers'],
    'top': ['top', 'blouse', 't-shirt', 'shirt', 'tshirt'],
    'shorts': ['shorts', 'swimshorts', 'short'],
    'skirt': ['skirt'],
    'headwear': ['cap', 'beanie', 'headband', 'hat'],
    'scarf & tie': ['scarf', 'tie', 'choker']
}
keyword2cat = {}
for k,v in cat2keywords.items() :
    for v_ in v :
        keyword2cat[v_] = k
keywords = list(keyword2cat.keys())

blacklist = ['pin', 'clogs', 'clog', 'strap', 
             'boxers', 'pack', 'rig', 
             'chain', 'bracelet', 'sandals', 'platform', 'swimsuit',
            'mules', 'bikini', 'keyring', 'sneaker', 'sneakers',
            'earring', 'boot', 'zip', 'briefs', 'socks', 'wallet',
             'towel', 'flats', 'moccasins', 'armband', 'pencils',
            'derbies', 'hi', 'cardholder', 'bodysuit', 'bra']
cats = list(cat2keywords.keys())

In [347]:
cat2fns = {}
for cat in cat2keywords.keys() :
    cat2fns[cat] = []
    
for fn in tqdm(farfetch_scrape_annos) :
    with open(fn, 'r') as f :
        anno = json.load(f)
    short_description = anno['short_description']
    if not short_description :
        continue
    short_split = short_description.replace('-', ' ').split()
    categories = [keyword2cat[w.lower()] for w in short_split if w.lower() in keywords]
    
    if len(categories) > 0 :
        category = categories[0]
        cat2fns[category].append(fn)
    else :
        long_desc = anno['description']
        if not long_desc :
            continue
            
        long_desc_split = long_desc.replace('-', ' ').split(' ')
        categories = [keyword2cat[w.lower()] for w in short_split if w.lower() in keywords]
        
        if len(categories) > 0 :
            category = categories[0]
            cat2fns[category].append(fn)

In [352]:
cat2num = pd.Series({k:len(v) for k,v in cat2fns.items()})

In [361]:
fns2cat = {}
for k,v in cat2fns.items() :
    for v_ in v :
        fns2cat[os.path.basename(v_).split('.')[0]] = k

In [364]:
with open('/data/farfetch-scrape-v2/product2category.json', 'w') as f :
    json.dump(fns2cat, f)